# **Pre-Processing Globalise**

Made by: Julius Salomons (14039559)  <br>
Date: 10-6-2025  <br>

A step by step guide for Preprocessing the datasets required to be used in the Final_Project.ipynb

In [39]:
# important library regularly used
import pandas as pd

# Used throughout for file and folder path handling: OUTPUT_DIR creation, checking file existence, deleting files, joining paths, directory checks
import os 

# Used in Step 1: list_dataset_txt_files (API requests), download_file (file download)
import requests  

# Used in Step 1: download_file (download progress bar), final loop over grouped years (progress bar for rows)
from tqdm import tqdm  
 
# Used in split_by_header_blocks (regex splits), normalize (text cleaning), extract_pages (extracting page numbers from strings), plus various string processing steps (multiple locations)
import re  

# Used in last big step for path management and file globbing (reading/writing csvs, directory operations)
from pathlib import Path  


import csv  
import sys  # Used in Step 4.2 for csv.field_size_limit and sys.exit() on fatal error (missing folder)


# **Step 1: Download and Convert Transcribed data "VOC v2 - GLOBALISE dataset"**

## **Step 1.1: Setup constants and output directory**

In [2]:
OUTPUT_DIR = './globalise_transcriptions_csv'
API_BASE = "https://datasets.iisg.amsterdam/api"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## **Step 1.2: List all .txt files from a dataset given its DOI**

In [3]:
def list_dataset_txt_files(doi):
    url = f"{API_BASE}/datasets/:persistentId"
    params = {"persistentId": doi}
    response = requests.get(url, params=params)
    response.raise_for_status()
    files = response.json()['data']['latestVersion']['files']
    return [
        {"id": f['dataFile']['id'], "label": f['label']}
        for f in files
        if f['label'].endswith('.txt')
    ]

## **Step 1.3: Download a file by its ID directly into the output directory**

In [4]:
def download_file(file_id, filename):
    local_path = os.path.join(OUTPUT_DIR, filename)
    url = f"{API_BASE}/access/datafile/{file_id}"
    print(f"Downloading: {filename}")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=8192), unit='KB'):
                f.write(chunk)
    return local_path


## **Step 1.4: Split text content based on headers**

In [5]:
def split_by_header_blocks(txt_content):
    parts = re.split(r"^\#\+ ---\s*$", txt_content, flags=re.MULTILINE)
    letters = []
    i = 0
    while i < len(parts) - 2:
        possible_header = parts[i + 1].strip()
        content = parts[i + 2].strip()
        if re.match(r"^\#\+\s+NL-HaNA_.*\.xml\s*$", possible_header):
            letters.append((possible_header, content))
            i += 2
        else:
            i += 1
    return letters


## **Step 1.5: Convert the list of letters into a pandas DataFrame**

In [6]:
def letters_to_dataframe(letters):
    return pd.DataFrame([{'Header': h, 'Content': c} for h, c in letters])

## **Step 1.6: Process one .txt file and save it as a CSV**

In [7]:
def process_and_save_csv(txt_file_path, csv_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        txt_content = f.read()

    letters = split_by_header_blocks(txt_content)
    df = letters_to_dataframe(letters)
    df['Content'] = df['Content'].apply(lambda x: '[NO CONTENT]' if not str(x).strip() else x)
    df['Header'] = df['Header'].apply(lambda x: pd.NA if not str(x).strip() else x)


    df.to_csv(csv_file_path, index=False)
    print(f"CSV-bestand opgeslagen als: {csv_file_path}")


## **Step 1.7: Main function to download and process all .txt files**

In [8]:
def download_and_process_txt_files(doi):
    txt_files = list_dataset_txt_files(doi)
    print(f"Gevonden {len(txt_files)} .txt bestanden.")

    for file_info in txt_files:
        filename = file_info["label"]
        csv_filename = os.path.splitext(filename)[0] + '_converted.csv'
        csv_file_path = os.path.join(OUTPUT_DIR, csv_filename)

        if os.path.exists(csv_file_path):
            print(f"CSV bestaat al, overslaan: {csv_filename}")
            continue

        txt_path = download_file(file_info["id"], filename)
        process_and_save_csv(txt_path, csv_file_path)
        os.remove(txt_path)  # Clean up .txt after processing


## **Step 1.8: Run the script for a specific dataset DOI**

Meant for: VOC v2 - GLOBALISE dataset DOI <br>
doi = "hdl:10622/LVXSBW" <br>
Runtime: roughly 40 min

In [9]:
doi = "hdl:10622/LVXSBW"
download_and_process_txt_files(doi)


Gevonden 6893 .txt bestanden.
Downloading: 10000.txt


45KB [00:00, 3245.86KB/s]


CSV-bestand opgeslagen als: ./globalise_transcriptions_csv\10000_converted.csv
Downloading: 10001.txt


21KB [00:00, 1600.97KB/s]


CSV-bestand opgeslagen als: ./globalise_transcriptions_csv\10001_converted.csv
Downloading: 10002.txt


...

CSV-bestand opgeslagen als: ./globalise_transcriptions_csv\9997_converted.csv
Downloading: 9998.txt


47KB [00:00, 932.63KB/s]


CSV-bestand opgeslagen als: ./globalise_transcriptions_csv\9998_converted.csv
Downloading: 9999.txt


120KB [00:00, 1099.69KB/s]


CSV-bestand opgeslagen als: ./globalise_transcriptions_csv\9999_converted.csv


# **Step 2: Extract and Prepare Generale Missive for GaLAHaD**

## **Step 2.1: Determine contents**

In [16]:
file_path = 'Overzicht van Generale Missiven.csv'
columns_of_interest = [
    'Bestandsnaam van eerste scan', 
    'Bestandsnaam van laatste scan',
    'Vroegste jaar', 
    'Laatste jaar'
]

try:
    missive_df = pd.read_csv(file_path, usecols=columns_of_interest)
    print("Metadata CSV loaded successfully.\n")
except FileNotFoundError:
    raise FileNotFoundError(f"File not found: {file_path}")
except Exception as e:
    raise RuntimeError(f"Failed to load CSV: {e}")

Metadata CSV loaded successfully.



## **Step 2.2: Extract File Number and Scan Info**

In [37]:
def extract_file_number(name):
    try:
        return name.split('_')[2]
    except Exception:
        return None

def extract_first_scan_number(name):
    try:
        return int(name.split('_')[3])
    except Exception:
        return None

def calculate_letter_count(start, end):
    try:
        return int(end.split('_')[3]) - int(start.split('_')[3]) + 1
    except Exception:
        return None

missive_df['File Number'] = missive_df['Bestandsnaam van eerste scan'].apply(extract_file_number)
missive_df['eerste scan'] = missive_df['Bestandsnaam van eerste scan'].apply(extract_first_scan_number)
missive_df['Aantal Brieven'] = missive_df.apply(lambda row: calculate_letter_count(
    row['Bestandsnaam van eerste scan'], row['Bestandsnaam van laatste scan']), axis=1)

# Rename date columns for clarity
missive_df.rename(columns={
    'Vroegste jaar': 'Start Date',
    'Laatste jaar': 'End Date'
}, inplace=True)

## **Step 2.3: Load and process each corresponding transcribed CSV file**

In [38]:
def process_and_combine_transcriptions(missive_df, folder_path):
    combined_contents = []

    for index, row in missive_df.iterrows():
        file_number = str(row['File Number'])
        eerste_scan = row['eerste scan']
        aantal_brieven = row['Aantal Brieven'] if pd.notna(row['Aantal Brieven']) else 1

        if pd.isna(file_number) or pd.isna(eerste_scan):
            combined_contents.append("Invalid file or scan info.")
            continue

        file_name = f"{file_number}_converted.csv"
        file_path = os.path.join(folder_path, file_name)

        if not os.path.exists(file_path):
            combined_contents.append(f"File {file_name} not found.")
            continue

        try:
            file_df = pd.read_csv(file_path)
        except Exception as e:
            combined_contents.append(f"Error reading {file_name}: {e}")
            continue

        if 'Content' not in file_df.columns:
            combined_contents.append(f"Missing 'Content' column in {file_name}")
            continue

        try:
            selected_rows = file_df.iloc[int(eerste_scan): int(eerste_scan) + int(aantal_brieven)]
        except Exception:
            combined_contents.append(f"Index error for {file_name}")
            continue

        combined_text = ""
        for i, content_row in selected_rows.iterrows():
            content = str(content_row['Content']).strip()
            if content:
                combined_text += f"\n--- Document {str(i).zfill(4)} ---\n{content}\n"

        combined_contents.append(combined_text.strip() if combined_text else "No non-empty content found.")

    missive_df['Generale Missive Content'] = combined_contents
    return missive_df



## **Step 2.4: Process and save the final unified CSV**

In [39]:
converted_csv_folder = 'globalise_transcriptions_csv'
missive_df = process_and_combine_transcriptions(missive_df, converted_csv_folder)

# Reorder and keep only relevant columns
final_columns = [
    'File Number', 'Start Date', 'End Date', 'Aantal Brieven', 
    'Generale Missive Content'
]
missive_df = missive_df[final_columns]

## **Step 2.5: Save final result**

In [40]:
final_output_path = os.path.join(converted_csv_folder, 'Transcribed_Missive.csv')
missive_df.to_csv(final_output_path, index=False)
print(f"\nFinal transcribed missive saved to: {final_output_path}")


Final transcribed missive saved to: globalise_transcriptions_csv\Transcribed_Missive.csv


# **Step 3: Translate Generale Missive via GaLAHaD**

In this step, the previously extracted .txt files will be translated and tagged using the external platform GaLAHaD. This process is performed manually through the online interface—no code is required.

### **Steps for Using GaLAHaD:**
**Step 3.1:** <br>
Access the GaLAHaD platform via: https://portal.clarin.ivdnt.org/galahad/home. Use valid credentials (e.g., University of Amsterdam login).

**Step 3.2:** <br>
Navigate to the "Annotate & Evaluate" tab.

**Step 3.3:** <br>
Under "Your corpora", click "New" and assign a name to your new corpus.

**Step 3.4:** <br>
Select the newly created corpus, then go to the "Documents" tab. Drag and drop the 921 Generale Missive .txt files and click "Upload".

**Step 3.5:** <br>
Go to the "Jobs" tab and select the tagger named "hug-tdn-1600-1900" (suitable for texts from 1600–1900).

**Step 3.6:** <br>
Click "View & Tag" and wait for the tagging and translation process to complete.

**Step 3.7:** <br>
Once finished, go to "Export", select the appropriate annotation layer (as in Step 3.5), and download the output as .tsv files.

**Step 3.8:** <br>
Place the downloaded .tsv files in a folder named "missive_15_6" to be used in Step 4.

### **Step 3.9: check if the files and folders are loaded in properly**

In [52]:
# Configuration
expected_tsv_count = 923
tsv_folder = 'missive_15_6'
txt_folder = 'missive_txt'

print("\nStep 3.9: Check GaLAHaD output for missing folders and/or files\n")

# Step 1: Check if TSV folder exists
if not os.path.isdir(tsv_folder):
    print(f"Folder not found: '{tsv_folder}'")
    print(f"Expected: {expected_tsv_count} .tsv files (e.g., missive_001.tsv to missive_{expected_tsv_count:03d}.tsv)")
    print("Please ensure the GaLAHaD export was completed and saved in this folder.\n")
else:
    print(f"TSV folder found: '{tsv_folder}'")
    tsv_files = sorted(f for f in os.listdir(tsv_folder) if f.endswith('.tsv'))
    actual_count = len(tsv_files)

    if actual_count == expected_tsv_count:
        print(f"All {expected_tsv_count} expected .tsv files are present. No missing files detected.\n")
    else:
        print(f"{actual_count} out of {expected_tsv_count} .tsv files found.")
        missing_count = expected_tsv_count - actual_count

        expected_filenames = {f"missive_{i:03d}.tsv" for i in range(1, expected_tsv_count + 1)}
        existing_filenames = set(tsv_files)
        missing_filenames = sorted(expected_filenames - existing_filenames)

        print(f"{len(missing_filenames)} .tsv files are missing from folder: '{tsv_folder}'.")

        # Step 2: Check if fallback TXT files exist
        if os.path.isdir(txt_folder):
            missing_txt = [
                tsv_name.replace('.tsv', '.txt')
                for tsv_name in missing_filenames
                if not os.path.isfile(os.path.join(txt_folder, tsv_name.replace('.tsv', '.txt')))
            ]

            if not missing_txt:
                print(f"All missing .tsv files have corresponding .txt fallback files available in folder: '{txt_folder}'.\n")
            else:
                print(f"{len(missing_txt)} of the missing .tsv files do not have corresponding .txt files in folder: '{txt_folder}'.")
        else:
            print(f"Fallback TXT folder not found: '{txt_folder}'")
            print("Unable to check for backup .txt files.\n")

print("Step 3.9 check complete.")


Step 3.9: Check GaLAHaD output for missing folders and/or files

TSV folder found: 'missive_15_6'
561 out of 923 .tsv files found.
362 .tsv files are missing from folder: 'missive_15_6'.
All missing .tsv files have corresponding .txt fallback files available in folder: 'missive_txt'.

Step 3.9 check complete.


# **Step 4: Extact Generale Missive from GaLAHaD**

## **Step 4.1: Normalization function for content in Generale Missive**

In [54]:
def normalize(text: str) -> str:
    if not isinstance(text, str):
        return ''
    
    # Normalize line endings
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    
    # Remove lines like "--- Document XXXX ---"
    text = re.sub(r'^--- Document \d+ ---\s*$', '', text, flags=re.MULTILINE)
    
    # Remove all characters except: letters, digits, whitespace, and sentence-ending punctuation
    text = re.sub(r'[^a-zA-Z0-9\s\.?!]+', '', text)
    
    # Collapse whitespace into a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

## **Step 4.2 (version A): Combines the MetaData for the Generale Missive with the translated text from the TSV files from GaLAHaD**

In [56]:
# Increase field size limit
csv.field_size_limit(sys.maxsize)

# Configuration
expected_files = 923
input_tsv_folder = 'missive_15_6'
metadata_file = 'Overzicht van Generale Missiven.csv'
output_csv_file = 'GaLAHaD_missive_complete.csv'

print("\nStep 4.2 (Version A): Start combining translated .tsv files with metadata")

# Check if TSV folder exists
if not os.path.isdir(input_tsv_folder):
    raise FileNotFoundError(f"The folder '{input_tsv_folder}' was not found. It should contain {expected_files} translated .tsv files from GaLAHaD.")

# Check for missing TSV files
tsv_files = {f for f in os.listdir(input_tsv_folder) if f.endswith('.tsv')}
expected_filenames = {f"missive_{i:03d}.tsv" for i in range(1, expected_files + 1)}
missing_tsvs = sorted(expected_filenames - tsv_files)

if missing_tsvs:
    print(f"\nError: Detected {len(missing_tsvs)} missing .tsv files in '{input_tsv_folder}'.")
    print("This version (A) requires a complete set of GaLAHaD-translated TSV files.")
    print("Please use Version B instead, which supports fallback to original TXT files.\n")
    sys.exit(1)

# Load metadata
meta_df = pd.read_csv(metadata_file)
if 'Laatste jaar' in meta_df.columns:
    meta_df['Laatste jaar'] = pd.to_numeric(meta_df['Laatste jaar'], errors='coerce').astype('Int64')
else:
    raise ValueError("Metadata file is missing the required column: 'Laatste jaar'.")

compiled_data = []

# Define normalize function (as in Version B or your previous)
def normalize(text: str) -> str:
    if not isinstance(text, str):
        return ''
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    text = re.sub(r'^--- Document \d+ ---\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z0-9\s\.?!]+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Process all TSV files
for i in range(1, expected_files + 1):
    file_id = f"{i:03d}"
    filename_tsv = f"missive_{file_id}.tsv"
    filepath_tsv = os.path.join(input_tsv_folder, filename_tsv)

    try:
        with open(filepath_tsv, 'r', encoding='utf-8') as tsv_file:
            reader = csv.DictReader(tsv_file, delimiter='\t')
            lemma_words = [
                row['lemma'] for row in reader
                if row.get('lemma') and row['lemma'].lower() != 'none'
            ]
            text_content = ' '.join(lemma_words).replace('  ', ' ').strip()
            normalized_text = normalize(text_content)
    except Exception as e:
        print(f"Error reading TSV file '{filename_tsv}': {e}")
        continue

    # Get year
    jaar = meta_df.loc[i - 1, 'Laatste jaar'] if i - 1 < len(meta_df) else pd.NA

    compiled_data.append({
        'filename': f"missive_{file_id}",
        'text': normalized_text,
        'year': jaar,
        'source': 'TSV'
    })

# Save to CSV
missives_df = pd.DataFrame(compiled_data)
missives_df.to_csv(output_csv_file, index=False, encoding='utf-8')
print(f"\nDone: {len(missives_df)} missives saved to '{output_csv_file}' (TSV only)\n")


Step 4.2 (Version A): Start combining translated .tsv files with metadata

Error: Detected 362 missing .tsv files in 'missive_15_6'.
This version (A) requires a complete set of GaLAHaD-translated TSV files.
Please use Version B instead, which supports fallback to original TXT files.



SystemExit: 1

C:\Users\juliu\AppData\Local\Programs\Python\Python313\Lib\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## **Step 4.2 (version B): Uses the Generale Missive original TXT files to suplement any missing TSV files**

This version Does essentially the same as version A except that whenever a TSV file appears to be missing it uses the original transcription data. This step was necessary since GaLAHaD was overworked and couldn't finish all the Generale Missive in time. 

In [57]:
csv.field_size_limit(sys.maxsize)

input_tsv_folder = 'missive_15_6'
input_txt_folder = 'missive_txt'
metadata_file = 'Overzicht van Generale Missiven.csv'
output_csv_file = 'GaLAHaD_missive_partial_translation.csv'

expected_files = 923
compiled_data = []

# Load metadata
meta_df = pd.read_csv(metadata_file)
if 'Laatste jaar' in meta_df.columns:
    meta_df['Laatste jaar'] = pd.to_numeric(meta_df['Laatste jaar'], errors='coerce').astype('Int64')
else:
    raise ValueError("Metadata file missing 'Laatste jaar' column.")

def normalize(text):
    # Replace multiple spaces with a single space
    text = ' '.join(text.split())
    # Remove space before punctuation marks (., !, ?)
    text = text.replace(' .', '.').replace(' ?', '?').replace(' !', '!')
    return text

for i in range(1, expected_files + 1):
    file_id = f"{i:03d}"  # e.g., 001
    filename_tsv = f"missive_{file_id}.tsv"
    filepath_tsv = os.path.join(input_tsv_folder, filename_tsv)

    filename_txt = f"missive_{file_id}.txt"
    filepath_txt = os.path.join(input_txt_folder, filename_txt)

    text_content = ""
    source_used = None

    # Try TSV first
    if os.path.isfile(filepath_tsv):
        try:
            with open(filepath_tsv, 'r', encoding='utf-8') as tsv_file:
                reader = csv.DictReader(tsv_file, delimiter='\t')
                lemma_words = []
                for row in reader:
                    lemma = row.get('lemma')
                    word = row.get('word')

                    if lemma and lemma.lower() != 'none':
                        lemma_words.append(lemma)
                    elif word in {'.', '?', '!'}:
                        lemma_words.append(word)
                    else:
                        continue  # Skip irrelevant punctuation or empty values

                text_content = ' '.join(lemma_words)
                # Fix spacing before punctuation
                text_content = text_content.replace(' .', '.').replace(' ?', '?').replace(' !', '!')
                text_content = normalize(text_content)
                source_used = 'TSV'
        except Exception as e:
            print(f"Error reading TSV {filename_tsv}: {e}")

    # Fallback to TXT
    elif os.path.isfile(filepath_txt):
        try:
            with open(filepath_txt, 'r', encoding='utf-8-sig') as txt_file:
                text_content = txt_file.read()

            if not text_content.strip():
                # Fallback to latin-1 if UTF-8 is empty
                with open(filepath_txt, 'r', encoding='latin-1') as txt_file:
                    text_content = txt_file.read()
                if text_content.strip():
                    print(f"Fallback encoding worked for {filename_txt}")
                else:
                    print(f"Empty after fallback: {filename_txt} (size: {os.path.getsize(filepath_txt)} bytes)")
            source_used = 'TXT'
        except Exception as e:
            print(f"Error reading TXT {filename_txt}: {e}")

    else:
        print(f"Missing both TSV and TXT for missive {file_id}")
        continue

    # Normalize text (already normalized above, but safe to repeat)
    normalized_text = normalize(text_content)

    # Get year from 'Overzicht van Generale Missiven.csv'
    jaar = meta_df.loc[i - 1, 'Laatste jaar'] if i - 1 < len(meta_df) else pd.NA

    # Store result
    compiled_data.append({
        'filename': f"missive_{file_id}",
        'text': normalized_text,
        'year': jaar,
        'source': source_used
    })

missives_df = pd.DataFrame(compiled_data)
missives_df.to_csv(output_csv_file, index=False, encoding='utf-8')
print(f"Done: {len(missives_df)} missives saved to '{output_csv_file}'")

Done: 923 missives saved to 'GaLAHaD_missive_partial_translation.csv'


# **Step 5: Sort transcribed data into files devided by years**

## **5.1: Convert NT00348_OBP.csv to dataframe**

In [10]:
import pandas as pd
from pathlib import Path

ref_file = Path("NT00348_OBP.csv")
df = pd.read_csv(ref_file, dtype=str)

columns_to_copy = [
    "vwz_inventarisnummer",
    "ove_datumtekst",
    "vwz_archivelink",
    "ove_pagina",
    "ove_archieftype",
    "ove_inhoud"
]

missing_cols = [col for col in columns_to_copy if col not in df.columns]
if missing_cols:
    raise KeyError(f"Expected columns {missing_cols} not found. Found columns: {df.columns.tolist()}")

df_extracted = df[columns_to_copy].copy()

## **5.2: Add 'end_page' column to Dataframe**

In [11]:
def extract_pages(archivelink: str):
    if not isinstance(archivelink, str):
        return None, None
    # Look for "Pagina 697-698" or "Pagina 697"
    match = re.search(r"Pagina\s+(\d+)(?:-(\d+))?", archivelink)
    if match:
        start_page = int(match.group(1))
        end_page = int(match.group(2)) if match.group(2) else start_page
        return start_page, end_page
    return None, None

# Apply to the dataframe
df_extracted['start_page'], df_extracted['end_page'] = zip(*df_extracted['vwz_archivelink'].apply(extract_pages))

# Convert columns to nullable integer dtype to avoid floats
df_extracted['start_page'] = df_extracted['start_page'].astype('Int64')
df_extracted['end_page'] = df_extracted['end_page'].astype('Int64')


### **5.2.1 (optional): Calculate amount of data in NT00348_OBP.csv versus transcriptions v2 dataset**

In [12]:
transcriptions_dir = Path("globalise_transcriptions_csv")

# Total pages from your existing code
total_pages = 0

for idx, row in df_extracted.iterrows():
    start = row['start_page']
    end = row['end_page']
    
    if pd.isna(start) or pd.isna(end):
        continue  # skip rows without valid pages
    
    total_pages += (end - start + 1)

print(f"Total pages counted (from NT00348_OBP.csv): {total_pages}")

total_rows = 0
file_count = 0
error_files = []

for file in transcriptions_dir.glob("*.csv"):
    try:
        df = pd.read_csv(file, encoding="utf-8")
        row_count = len(df)
        total_rows += row_count
        file_count += 1
    except Exception as e:
        print(f"[Error reading] {file.name}: {e}")
        error_files.append(file.name)

print(f"Total rows across {file_count} transcription files: {total_rows}")

# Loss calculation
if total_pages > 0:
    lost = total_pages - total_rows
    pct_lost = (lost / total_pages) * 100
    print("\nData coverage summary:")
    print(f"Pages expected         : {total_pages}")
    print(f"Rows available         : {total_rows}")
    print(f"Pages/rows missing     : {lost}")
    print(f"Percentage missing     : {pct_lost:.2f}%")
else:
    print("No valid page data found to compare.")

if error_files:
    print(f"\nFiles with read errors ({len(error_files)}):")
    for fname in error_files:
        print(f"- {fname}")

Total pages counted (from metadata): 2370424
Total rows across 6894 transcription files: 4786958

Data coverage summary:
Pages expected         : 2370424
Rows available         : 4786958
Pages/rows missing     : -2416534
Percentage missing     : -101.95%


## **5.3: Finalize and sort the DataFrame**

In [51]:
def expand_years(df, ove_datumtekst='ove_datumtekst', sep=','):
    # Split the year column by separator into lists
    split_years = df[ove_datumtekst].str.split(sep)
    
    # Repeat rows for the length of the split years
    df_expanded = df.loc[df.index.repeat(split_years.str.len())].copy()
    df_expanded[ove_datumtekst] = [y for sublist in split_years for y in sublist]
    df_expanded[ove_datumtekst] = df_expanded[ove_datumtekst].str.strip()
    df_expanded = df_expanded.reset_index(drop=True)
    
    return df_expanded

# Apply the expansion
df_expanded = expand_years(df_extracted, ove_datumtekst='ove_datumtekst', sep=',')

# Convert 'ove_datumtekst' to integer for sorting
df_expanded['ove_datumtekst'] = df_expanded['ove_datumtekst'].astype(int)

# Sort by the year column ascending (oldest to newest)
df_expanded = df_expanded.sort_values(by='ove_datumtekst').reset_index(drop=True)

## **5.4: Convert DataFrame to csv file 'vwz_expanded_sorted.CSV'**

In [52]:
# Save the sorted dataframe
output_file = Path("vwz_expanded_sorted.csv")
df_expanded.to_csv(output_file, index=False)

print(f"Saved sorted {len(df_expanded)} rows to {output_file}")

Saved sorted 191562 rows to vwz_expanded_sorted.csv


## **5.5: Create one csv file for every year and add contents (from globalise_transcriptions_csv)**

Runtime: 2+ hours

In [13]:
input_file = Path("vwz_expanded_sorted.csv")
output_dir = Path("transcriptions_sorted")
transcriptions_dir = Path("globalise_transcriptions_csv")

output_dir.mkdir(exist_ok=True)

# Load the main dataframe
df = pd.read_csv(input_file, dtype={"ove_datumtekst": int, "start_page": pd.Int64Dtype(), "end_page": pd.Int64Dtype()})

# Columns to keep in output CSVs
columns_to_keep = ["vwz_inventarisnummer", "vwz_archivelink", "ove_archieftype", "ove_inhoud"]

# Group by year
for year, group in df.groupby('ove_datumtekst'):
    rows_for_year = []

    # Progress bar for rows in this year
    for _, row in tqdm(group.iterrows(), total=len(group), desc=f"Year {year}"):
        inv_num = row['vwz_inventarisnummer']
        start_page = row['start_page']
        end_page = row['end_page']

        # Calculate total pages
        if pd.isna(start_page) or pd.isna(end_page):
            total_pages = 0
        else:
            total_pages = end_page - start_page + 1

        combined_content = ""

        trans_file = transcriptions_dir / f"{inv_num}_converted.csv"
        if trans_file.exists():
            try:
                df_trans = pd.read_csv(trans_file)

                # Use row index slicing assuming 1-based page numbers
                if not pd.isna(start_page) and not pd.isna(end_page):
                    try:
                        selected_rows = df_trans.iloc[start_page - 1:end_page]
                        if 'Content' in selected_rows.columns:
                            contents = selected_rows['Content'].dropna().astype(str)
                            combined_content = "\n".join(contents)
                    except IndexError:
                        pass  # silently skip index issues
            except Exception:
                pass  # silently skip read errors

        new_row = {col: row[col] for col in columns_to_keep}
        new_row['total_pages'] = total_pages
        new_row['content'] = combined_content

        # Skip rows with "[NO CONTENT]" or empty content
        if combined_content.strip() == "[NO CONTENT]" or combined_content.strip() == "":
            continue

        rows_for_year.append(new_row)

    # Save year-specific file
    df_year = pd.DataFrame(rows_for_year)
    year_file = output_dir / f"{year}.csv"
    df_year.to_csv(year_file, index=False, encoding='utf-8')



Year 1983: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.25it/s]
